![spotify_logo](images/spotify_logo2.1.png)

# Spotify API Scrape

## Project Goal
1. Pull most recently played tracks
2. Pull top listen to tracks

## Helpful Links:
- [Spotify Web API - Authorization Guide](https://developer.spotify.com/documentation/general/guides/authorization-guide/)
- [Spotify API References](https://developer.spotify.com/documentation/web-api/reference/)

In [1]:
import config
import os
import requests
import json
from json import JSONEncoder
import pandas as pd
from datetime import datetime,timezone, timedelta
from tqdm import tqdm
import numpy as np
import spotipy
from spotipy.oauth2 import SpotifyOAuth
import base64
from urllib.parse import urlencode

import pytz
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy.util as util
import xlsxwriter
from tzlocal import get_localzone
import dateutil

In [2]:
# Create the file name we will be exporting later
today = datetime.today().strftime('%Y%m%d')
file_name = f"Spotify_Export_{today}.xlsx"
file_name

'Spotify_Export_20200824.xlsx'

## Step 1: Get Access Token

In [3]:
client_id = config.client_id
client_secret = config.client_secret
username = config.username

In [4]:
username = config.username
client_id = config.client_id
client_secret = config.client_secret
redirect_uri = 'http://localhost:7777/callback'
scope = 'user-read-recently-played'

auth_token = util.prompt_for_user_token(username=username, 
                                   scope=scope, 
                                   client_id=client_id,   
                                   client_secret=client_secret,     
                                   redirect_uri=redirect_uri)

## Step 2: Pull Recently Played

In [5]:
base_url = 'https://api.spotify.com/v1/me/player/recently-played?'
#track_id = '6y0igZArWVi6Iz0rj35c1Y'

#2. Authentication
#3. Parameters -- would be stored with authentication
headers = {
    "Authorization": f"Bearer {auth_token}"
}

#4. Create an empty list
personal_data = [] #would be good explore how to capture data at different points in time
r = requests.get(base_url+"&limit=50", headers=headers)
personal_data.append(json.loads(r.text))

In [6]:
personal_data[0]['items'][3]['track']['artists'][0]['id']

'6lcBiGiT3dlyDMjBBtfyfS'

In [7]:
track_ids = []
album_ids = []
artist_ids = []
album_names = []
artist_names = []
track_names = []
popularity_ls = []
played_ats = []

for i in range(len(personal_data[0]['items'])):
    track_ids.append(personal_data[0]['items'][i]['track']['id']) # Track ID
    album_ids.append(personal_data[0]['items'][i]['track']['album']['id']) # Albumn ID
    artist_ids.append(personal_data[0]['items'][i]['track']['artists'][0]['id']) # Artist ID
    album_names.append(personal_data[0]['items'][i]['track']['album']['name']) # Album Name
    artist_names.append(personal_data[0]['items'][i]['track']['artists'][0]['name']) # Artist Name
    track_names.append(personal_data[0]['items'][i]['track']['name']) # Track Name
    popularity_ls.append(personal_data[0]['items'][i]['track']['popularity']) # Track Popularity
    played_ats.append(personal_data[0]['items'][i]['played_at'])

In [8]:
list_dic={'track_id':track_ids,
          'album_id':album_ids,
          'artist_id':artist_ids,
          'track_name':track_names,
          'artist_name':artist_names,
          'album_name':album_names,
          'track_popularity':popularity_ls,
          'time_palyed': played_ats
    }

In [9]:
df1=pd.DataFrame(list_dic)
df1

,track_id,album_id,artist_id,track_name,artist_name,album_name,track_popularity,time_palyed
0,2zmDSwptUhsGSkPc4fcHTK,2yvPg4ArTtYi2YsdCwzxoI,7F5gkuNk8gKIYIQtXJNFAw,Pocket Change,Secondhand Sound,Burnout,32,2020-08-25T00:04:02.710Z
1,78ZtB8NKA4ClU3QNBagmk5,2G8nSBJoezctZiZdIj0Aq6,52G6EB7xvVYA8IzF1T335g,Tie Me Up,Micky James,Tie Me Up & New Heart,37,2020-08-24T23:58:21.030Z
2,1FDHJ57pYuT4MsQuwHaRci,7erd3cFfa9yCboGPYaJ2Yh,4vxaQs6vK54nK89J1VtLex,Make You Mine (feat. MisterWives),PUBLIC,Make You Mine (feat. MisterWives),41,2020-08-24T23:55:48.465Z
3,3XrnghYVpHVCl3dRuCSjIs,0SrAyncfmgxAafsVe8DGVU,6lcBiGiT3dlyDMjBBtfyfS,Apt. 4,Vacation Manor,Apt. 4,43,2020-08-24T23:52:05.907Z
4,1UU5XKB9gMQd5lGAFHjYo6,0NGWQlhXnZPGXVvdBhRuX5,2ryTEYHBsD0Ks4nytV1wle,Overboard,Stay Outside,Overboard,37,2020-08-24T23:48:58.354Z
5,1cGt47ods2UZyDicTwKmPP,17JFJfZqZ8AFgVRr89lqAT,0INDB6Snts5NDbzh8jC3lk,Salt Pond - Bonus Track,Run River North,Superstition EP,20,2020-08-24T22:57:41.765Z
6,78ZtB8NKA4ClU3QNBagmk5,2G8nSBJoezctZiZdIj0Aq6,52G6EB7xvVYA8IzF1T335g,Tie Me Up,Micky James,Tie Me Up & New Heart,37,2020-08-24T22:35:37.145Z
7,1FDHJ57pYuT4MsQuwHaRci,7erd3cFfa9yCboGPYaJ2Yh,4vxaQs6vK54nK89J1VtLex,Make You Mine (feat. MisterWives),PUBLIC,Make You Mine (feat. MisterWives),41,2020-08-24T22:33:04.623Z
8,3XrnghYVpHVCl3dRuCSjIs,0SrAyncfmgxAafsVe8DGVU,6lcBiGiT3dlyDMjBBtfyfS,Apt. 4,Vacation Manor,Apt. 4,43,2020-08-24T22:29:22.615Z
9,1UU5XKB9gMQd5lGAFHjYo6,0NGWQlhXnZPGXVvdBhRuX5,2ryTEYHBsD0Ks4nytV1wle,Overboard,Stay Outside,Overboard,37,2020-08-24T22:24:14.263Z


In [10]:
# Create empty column to append data to
df1['local_time'] = ''

for i in range(len(personal_data[0]['items'])):
    # Convert UTC to local time zone
    utc_time = dateutil.parser.parse(df1['time_palyed'].iloc[i]).astimezone(get_localzone())
    # Format date/time
    local_time= utc_time.strftime('%Y-%m-%d %H:%M:%S')
    local_time
    
    df1['local_time'].iloc[i] = local_time

C:\Users\Nicholas\Anaconda3\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [11]:
df1.head()

,track_id,album_id,artist_id,track_name,artist_name,album_name,track_popularity,time_palyed,local_time
0,2zmDSwptUhsGSkPc4fcHTK,2yvPg4ArTtYi2YsdCwzxoI,7F5gkuNk8gKIYIQtXJNFAw,Pocket Change,Secondhand Sound,Burnout,32,2020-08-25T00:04:02.710Z,2020-08-24 19:04:02
1,78ZtB8NKA4ClU3QNBagmk5,2G8nSBJoezctZiZdIj0Aq6,52G6EB7xvVYA8IzF1T335g,Tie Me Up,Micky James,Tie Me Up & New Heart,37,2020-08-24T23:58:21.030Z,2020-08-24 18:58:21
2,1FDHJ57pYuT4MsQuwHaRci,7erd3cFfa9yCboGPYaJ2Yh,4vxaQs6vK54nK89J1VtLex,Make You Mine (feat. MisterWives),PUBLIC,Make You Mine (feat. MisterWives),41,2020-08-24T23:55:48.465Z,2020-08-24 18:55:48
3,3XrnghYVpHVCl3dRuCSjIs,0SrAyncfmgxAafsVe8DGVU,6lcBiGiT3dlyDMjBBtfyfS,Apt. 4,Vacation Manor,Apt. 4,43,2020-08-24T23:52:05.907Z,2020-08-24 18:52:05
4,1UU5XKB9gMQd5lGAFHjYo6,0NGWQlhXnZPGXVvdBhRuX5,2ryTEYHBsD0Ks4nytV1wle,Overboard,Stay Outside,Overboard,37,2020-08-24T23:48:58.354Z,2020-08-24 18:48:58


In [12]:
writer = pd.ExcelWriter(file_name)
df1.to_excel(writer,'recently_played')

## Step 3: Pull User's Top Artists & Tracks

In [13]:
username = config.username
client_id = config.client_id
client_secret = config.client_secret
redirect_uri = 'http://localhost:7777/callback'
scope = 'user-top-read'

auth_token = util.prompt_for_user_token(username=username, 
                                   scope=scope, 
                                   client_id=client_id,   
                                   client_secret=client_secret,     
                                   redirect_uri=redirect_uri)

In [14]:
base_url = 'https://api.spotify.com/v1/me/top/tracks?'
#track_id = '6y0igZArWVi6Iz0rj35c1Y'

#2. Authentication
#3. Parameters -- would be stored with authentication
headers = {
    "Authorization": f"Bearer {auth_token}"
}

#4. Create an empty list
top_track_data = [] #would be good explore how to capture data at different points in time
r = requests.get(base_url+"time_range=medium_term"+"&&limit=50", headers=headers)
top_track_data.append(json.loads(r.text))

In [15]:
track_idss = []
album_idss = []
artist_idss = []
album_namess = []
album_relase_datess = []
artist_namess = []
popularity_lss = []
track_namess = []

for i in range(len(top_track_data[0]['items'])):
    track_idss.append(top_track_data[0]['items'][i]['id']) # Track ID
    album_idss.append(top_track_data[0]['items'][i]['album']['id']) # Album ID
    artist_idss.append(top_track_data[0]['items'][i]['album']['artists'][0]['id'])
    album_namess.append(top_track_data[0]['items'][i]['album']['name']) # Album Name
    album_relase_datess.append(top_track_data[0]['items'][i]['album']['release_date'])
    artist_namess.append(top_track_data[0]['items'][i]['album']['artists'][0]['name']) # Artist Name
    popularity_lss.append(top_track_data[0]['items'][i]['popularity'])
    track_namess.append(top_track_data[0]['items'][i]['name']) # Track Name

In [16]:
list_dic2={'track_id':track_idss,
           'album_id':album_idss,
           'artist_id':artist_idss,
           'track_name':track_namess,
           'album_name':album_namess,
           'artist_name':artist_namess,
           'track_popularity':popularity_lss,
           'album_relase_date':album_relase_datess,
    }

In [17]:
df2=pd.DataFrame(list_dic2)
df2

,track_id,album_id,artist_id,track_name,album_name,artist_name,track_popularity,album_relase_date
0,4pvb0WLRcMtbPGmtejJJ6y,2fenSS68JI1h4Fo296JfGr,06HL4z0CvFAxyc27GXpf02,exile (feat. Bon Iver),folklore,Taylor Swift,92,2020-07-24
1,6xZ4Q2k2ompmDppyeESIY8,4h3HXlnt6lryGzGbWmcFuY,3YQKmKGau1PzlVlkL1iodx,Level of Concern,Level of Concern,Twenty One Pilots,81,2020-04-09
2,1ci0BoqpvH73L2TJzHhw9y,3FallGXQGmURWo6JW3k1gM,3hSFS64223jyO9Ck66rLOf,Modern Chemistry,Tell All Your Friend,Okey Dokey,44,2019-03-22
3,4w2tfK0JA8KrVegKnxukf4,4XOMJHVuzJVWmqUdp4SYKP,5tFRohaO5yEsuJxmMnlCO9,The Kids Are Alright,404,Barns Courtney,38,2019-09-06
4,0VjIjW4GlUZAMYd2vXMi3b,4yP0hdKOZPNshxUOjY0cZj,1Xyo4u8uXC1ZmMpatF05PJ,Blinding Lights,After Hours,The Weeknd,99,2020-03-20
5,6xQpOC55lufcqXvSzp7GTb,7c2VhL8HYMCwWIs54xEX2o,5tFRohaO5yEsuJxmMnlCO9,Hard To Be Alone,Hard To Be Alone,Barns Courtney,42,2020-07-10
6,1uddOsj7TyRA13hnS2yDyk,2bMqAaEuEUDHBkdTNAUl1f,21MZoYDTcbPsC5crOQVqho,Can't Take My Eyes Off You,Can't Take My Eyes Off You / 3BadSoSad - Edit,Private Island,47,2019-11-15
7,2nUV1fiD45RN6cQZ85GDc1,0lvgNBfe5wm9gKkecWN5Hq,18qigp8zUYz9THL4t4vZV4,Salt (Nlmg),The Light Left Over,Ben Hon,39,2019-04-05
8,3FRJFImdfX5NSY3QH3jI4u,4cZhiGqIqqqlHxWOC9r7Jp,7FKTg75ADVMZgY3P9ZMRtH,Glistening,Something You Needed,Flipturn,37,2020-02-07
9,3flgdcFBWI84DPi4s1jhhd,4cZhiGqIqqqlHxWOC9r7Jp,7FKTg75ADVMZgY3P9ZMRtH,Savannah,Something You Needed,Flipturn,30,2020-02-07


## Step 4: Pull Track Info
- API Doc: https://developer.spotify.com/documentation/web-api/reference/tracks/get-audio-analysis/

In [18]:
# Create list of track_ids
track_id_ls = df2['track_id'].tolist()

In [19]:
track_data = []
for id in tqdm(track_id_ls):
    base_url = f'https://api.spotify.com/v1/audio-features/{id}?'

    #2. Authentication
    #3. Parameters -- would be stored with authentication
    headers = {
        "Authorization": f"Bearer {auth_token}"
    }

    r = requests.get(base_url, headers=headers)
    track_data.append(json.loads(r.text))

100%|██████████| 50/50 [00:04<00:00, 11.70it/s]


In [20]:
track_df = pd.json_normalize(track_data)
track_df.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.298,0.380,6,-8.426,1,0.0287,0.77800,0.000056,0.1100,0.152,75.602,audio_features,4pvb0WLRcMtbPGmtejJJ6y,spotify:track:4pvb0WLRcMtbPGmtejJJ6y,https://api.spotify.com/v1/tracks/4pvb0WLRcMtb...,https://api.spotify.com/v1/audio-analysis/4pvb...,285634,4
1,0.754,0.583,4,-7.340,0,0.0432,0.32000,0.000150,0.1440,0.770,122.012,audio_features,6xZ4Q2k2ompmDppyeESIY8,spotify:track:6xZ4Q2k2ompmDppyeESIY8,https://api.spotify.com/v1/tracks/6xZ4Q2k2ompm...,https://api.spotify.com/v1/audio-analysis/6xZ4...,220051,4
2,0.548,0.608,5,-7.506,1,0.0265,0.00415,0.000025,0.0665,0.436,89.997,audio_features,1ci0BoqpvH73L2TJzHhw9y,spotify:track:1ci0BoqpvH73L2TJzHhw9y,https://api.spotify.com/v1/tracks/1ci0BoqpvH73...,https://api.spotify.com/v1/audio-analysis/1ci0...,230213,4
3,0.547,0.771,3,-4.722,1,0.0284,0.02660,0.000001,0.0819,0.674,122.036,audio_features,4w2tfK0JA8KrVegKnxukf4,spotify:track:4w2tfK0JA8KrVegKnxukf4,https://api.spotify.com/v1/tracks/4w2tfK0JA8Kr...,https://api.spotify.com/v1/audio-analysis/4w2t...,235773,4
4,0.514,0.730,1,-5.934,1,0.0598,0.00146,0.000095,0.0897,0.334,171.005,audio_features,0VjIjW4GlUZAMYd2vXMi3b,spotify:track:0VjIjW4GlUZAMYd2vXMi3b,https://api.spotify.com/v1/tracks/0VjIjW4GlUZA...,https://api.spotify.com/v1/audio-analysis/0VjI...,200040,4


## Step 5: Pull Artist Info
- [Artist Endpoint Documentation](https://developer.spotify.com/documentation/web-api/reference/artists/get-artist/)

In [21]:
# Creating a uniqe list artist ids
artist_id_ls = df2['artist_id'].tolist()
artist_id_ls = list(dict.fromkeys(artist_id_ls))

artist_id_ls

['06HL4z0CvFAxyc27GXpf02',
 '3YQKmKGau1PzlVlkL1iodx',
 '3hSFS64223jyO9Ck66rLOf',
 '5tFRohaO5yEsuJxmMnlCO9',
 '1Xyo4u8uXC1ZmMpatF05PJ',
 '21MZoYDTcbPsC5crOQVqho',
 '18qigp8zUYz9THL4t4vZV4',
 '7FKTg75ADVMZgY3P9ZMRtH',
 '7gRH7pJQJaAvrwvi8STYEl',
 '2D4FOOOtWycb3Aw9nY5n3c',
 '0sBxk1gyQM4VN1j1pS01GJ',
 '0epOFNiUfyON9EYx7Tpr6V',
 '6eJa3zG1QZLRB3xgRuyxbm',
 '5cVeSOiS002MF1uiUFOPV5',
 '4ug3P1K8BaCdJXROrqHqhu',
 '4YZ5ECfbM2xSTSQTJGBbO5',
 '4njdEjTnLfcGImKZu1iSrz',
 '7mnBLXK823vNxN3UWB7Gfz',
 '7gP3bB2nilZXLfPHJhMdvc',
 '2PKUc5LXsZNjqOpAZKcFgz',
 '3tiDO34wzfyJdv8elYqKxj']

In [22]:
artist_data = []

for a_id in tqdm(artist_id_ls):
    base_url = f'https://api.spotify.com/v1/artists/{a_id}?'
    # example artist_id = '06HL4z0CvFAxyc27GXpf02'

    #2. Authentication
    #3. Parameters -- would be stored with authentication
    headers = {
        "Authorization": f"Bearer {auth_token}"
    }

    #4. Create an empty list
    r = requests.get(base_url, headers=headers)
    artist_data.append(json.loads(r.text))

100%|██████████| 21/21 [00:01<00:00, 14.33it/s]


In [23]:
artist_data

[{'external_urls': {'spotify': 'https://open.spotify.com/artist/06HL4z0CvFAxyc27GXpf02'},
  'followers': {'href': None, 'total': 31706960},
  'genres': ['pop'],
  'href': 'https://api.spotify.com/v1/artists/06HL4z0CvFAxyc27GXpf02',
  'id': '06HL4z0CvFAxyc27GXpf02',
  'images': [{'height': 640,
    'url': 'https://i.scdn.co/image/a37efbc7fd3f5f5df81b48ce9c6de53820b239c1',
    'width': 640},
   {'height': 320,
    'url': 'https://i.scdn.co/image/dc6f0ac18ad5db302e3c8a28b655b6ca7244ef08',
    'width': 320},
   {'height': 160,
    'url': 'https://i.scdn.co/image/b1627ca86690d2c5b6cc2fb1039a31014e96a22b',
    'width': 160}],
  'name': 'Taylor Swift',
  'popularity': 98,
  'type': 'artist',
  'uri': 'spotify:artist:06HL4z0CvFAxyc27GXpf02'},
 {'external_urls': {'spotify': 'https://open.spotify.com/artist/3YQKmKGau1PzlVlkL1iodx'},
  'followers': {'href': None, 'total': 18044488},
  'genres': ['modern rock', 'rock'],
  'href': 'https://api.spotify.com/v1/artists/3YQKmKGau1PzlVlkL1iodx',
  'id':

In [24]:
len(artist_data[7]['genres'])

1

In [25]:
artist_id_ls = []
artist_followers_ls = []
artist_genres_ls = []
artist_name_ls = []
artist_popularity_ls = []

for i in tqdm(range(len(artist_data))):
    artist_id_ls.append(artist_data[i]['id'])
    artist_followers_ls.append(artist_data[i]['followers']['total'])
    artist_name_ls.append(artist_data[i]['name'])
    artist_popularity_ls.append(artist_data[i]['popularity'])
    
    if len(artist_data[i]['genres']) > 0:
        artist_genres_ls.append(artist_data[i]['genres'][0])
    else:
        artist_genres_ls.append('not listed')

100%|██████████| 21/21 [00:00<00:00, 21016.56it/s]


In [26]:
list_dic3={'artist_id':artist_id_ls,
           'artist_followers':artist_followers_ls,
           'artist_genres':artist_genres_ls,
           'artist_name':artist_name_ls,
           'artist_popularity':artist_popularity_ls
    }

In [27]:
artist_df = pd.DataFrame(list_dic3)
artist_df

,artist_id,artist_followers,artist_genres,artist_name,artist_popularity
0,06HL4z0CvFAxyc27GXpf02,31706960,pop,Taylor Swift,98
1,3YQKmKGau1PzlVlkL1iodx,18044488,modern rock,Twenty One Pilots,85
2,3hSFS64223jyO9Ck66rLOf,20837,indie garage rock,Okey Dokey,47
3,5tFRohaO5yEsuJxmMnlCO9,323721,modern alternative rock,Barns Courtney,66
4,1Xyo4u8uXC1ZmMpatF05PJ,23502363,canadian contemporary r&b,The Weeknd,95
5,21MZoYDTcbPsC5crOQVqho,24552,vapor soul,Private Island,46
6,18qigp8zUYz9THL4t4vZV4,366,not listed,Ben Hon,25
7,7FKTg75ADVMZgY3P9ZMRtH,18915,jacksonville indie,Flipturn,49
8,7gRH7pJQJaAvrwvi8STYEl,13324,lexington ky indie,Matt Duncan,45
9,2D4FOOOtWycb3Aw9nY5n3c,565502,indie pop,Declan McKenna,69


## Clean Up Track Data

## Update 'mode' to tell if track is major or minor

In [28]:
mod_dict = {0 : 'Minor',
            1: 'Major'}

In [29]:
track_df['mode'].replace(mod_dict, inplace=True)

## Update "key" to tell the actually key

In [30]:
music_dic = {
    0: 'C',
    1: 'C#/Db',
    2: 'D',
    3: 'D#/Eb',
    4: 'E',
    5: 'F',
    6: 'F#/Gb',
    7: 'G',
    8: 'G#/Ab',
    9: 'A',
    10: 'A#/Bb',
    11: 'B' 
}

In [31]:
track_df['key'].replace(music_dic, inplace=True)

## Add column duration in minutes/seconds

In [32]:
track_df['track_duration'] = ''

for i in range(len(track_df['id'])):
    millis=track_df['duration_ms'].iloc[i]
    track_df['track_duration'].iloc[i] = pd.to_datetime(millis, unit='ms').strftime('%H:%M:%S')

C:\Users\Nicholas\Anaconda3\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


## Rename track id column

In [33]:
track_df=track_df.rename(columns = {'id':'track_id'})
track_df.head()                     

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,track_id,uri,track_href,analysis_url,duration_ms,time_signature,track_duration
0,0.298,0.380,F#/Gb,-8.426,Major,0.0287,0.77800,0.000056,0.1100,0.152,75.602,audio_features,4pvb0WLRcMtbPGmtejJJ6y,spotify:track:4pvb0WLRcMtbPGmtejJJ6y,https://api.spotify.com/v1/tracks/4pvb0WLRcMtb...,https://api.spotify.com/v1/audio-analysis/4pvb...,285634,4,00:04:45
1,0.754,0.583,E,-7.340,Minor,0.0432,0.32000,0.000150,0.1440,0.770,122.012,audio_features,6xZ4Q2k2ompmDppyeESIY8,spotify:track:6xZ4Q2k2ompmDppyeESIY8,https://api.spotify.com/v1/tracks/6xZ4Q2k2ompm...,https://api.spotify.com/v1/audio-analysis/6xZ4...,220051,4,00:03:40
2,0.548,0.608,F,-7.506,Major,0.0265,0.00415,0.000025,0.0665,0.436,89.997,audio_features,1ci0BoqpvH73L2TJzHhw9y,spotify:track:1ci0BoqpvH73L2TJzHhw9y,https://api.spotify.com/v1/tracks/1ci0BoqpvH73...,https://api.spotify.com/v1/audio-analysis/1ci0...,230213,4,00:03:50
3,0.547,0.771,D#/Eb,-4.722,Major,0.0284,0.02660,0.000001,0.0819,0.674,122.036,audio_features,4w2tfK0JA8KrVegKnxukf4,spotify:track:4w2tfK0JA8KrVegKnxukf4,https://api.spotify.com/v1/tracks/4w2tfK0JA8Kr...,https://api.spotify.com/v1/audio-analysis/4w2t...,235773,4,00:03:55
4,0.514,0.730,C#/Db,-5.934,Major,0.0598,0.00146,0.000095,0.0897,0.334,171.005,audio_features,0VjIjW4GlUZAMYd2vXMi3b,spotify:track:0VjIjW4GlUZAMYd2vXMi3b,https://api.spotify.com/v1/tracks/0VjIjW4GlUZA...,https://api.spotify.com/v1/audio-analysis/0VjI...,200040,4,00:03:20


# Create Master Dataframe

In [34]:
master_df = pd.merge(df2, track_df,
                       how='left', on=['track_id'])

In [35]:
master_df.head(10)

,track_id,album_id,artist_id,track_name,album_name,artist_name,track_popularity,album_relase_date,danceability,energy,...,liveness,valence,tempo,type,uri,track_href,analysis_url,duration_ms,time_signature,track_duration
0,4pvb0WLRcMtbPGmtejJJ6y,2fenSS68JI1h4Fo296JfGr,06HL4z0CvFAxyc27GXpf02,exile (feat. Bon Iver),folklore,Taylor Swift,92,2020-07-24,0.298,0.3800,...,0.1100,0.152,75.602,audio_features,spotify:track:4pvb0WLRcMtbPGmtejJJ6y,https://api.spotify.com/v1/tracks/4pvb0WLRcMtb...,https://api.spotify.com/v1/audio-analysis/4pvb...,285634,4,00:04:45
1,6xZ4Q2k2ompmDppyeESIY8,4h3HXlnt6lryGzGbWmcFuY,3YQKmKGau1PzlVlkL1iodx,Level of Concern,Level of Concern,Twenty One Pilots,81,2020-04-09,0.754,0.5830,...,0.1440,0.770,122.012,audio_features,spotify:track:6xZ4Q2k2ompmDppyeESIY8,https://api.spotify.com/v1/tracks/6xZ4Q2k2ompm...,https://api.spotify.com/v1/audio-analysis/6xZ4...,220051,4,00:03:40
2,1ci0BoqpvH73L2TJzHhw9y,3FallGXQGmURWo6JW3k1gM,3hSFS64223jyO9Ck66rLOf,Modern Chemistry,Tell All Your Friend,Okey Dokey,44,2019-03-22,0.548,0.6080,...,0.0665,0.436,89.997,audio_features,spotify:track:1ci0BoqpvH73L2TJzHhw9y,https://api.spotify.com/v1/tracks/1ci0BoqpvH73...,https://api.spotify.com/v1/audio-analysis/1ci0...,230213,4,00:03:50
3,4w2tfK0JA8KrVegKnxukf4,4XOMJHVuzJVWmqUdp4SYKP,5tFRohaO5yEsuJxmMnlCO9,The Kids Are Alright,404,Barns Courtney,38,2019-09-06,0.547,0.7710,...,0.0819,0.674,122.036,audio_features,spotify:track:4w2tfK0JA8KrVegKnxukf4,https://api.spotify.com/v1/tracks/4w2tfK0JA8Kr...,https://api.spotify.com/v1/audio-analysis/4w2t...,235773,4,00:03:55
4,0VjIjW4GlUZAMYd2vXMi3b,4yP0hdKOZPNshxUOjY0cZj,1Xyo4u8uXC1ZmMpatF05PJ,Blinding Lights,After Hours,The Weeknd,99,2020-03-20,0.514,0.7300,...,0.0897,0.334,171.005,audio_features,spotify:track:0VjIjW4GlUZAMYd2vXMi3b,https://api.spotify.com/v1/tracks/0VjIjW4GlUZA...,https://api.spotify.com/v1/audio-analysis/0VjI...,200040,4,00:03:20
5,6xQpOC55lufcqXvSzp7GTb,7c2VhL8HYMCwWIs54xEX2o,5tFRohaO5yEsuJxmMnlCO9,Hard To Be Alone,Hard To Be Alone,Barns Courtney,42,2020-07-10,0.414,0.4760,...,0.0863,0.240,94.087,audio_features,spotify:track:6xQpOC55lufcqXvSzp7GTb,https://api.spotify.com/v1/tracks/6xQpOC55lufc...,https://api.spotify.com/v1/audio-analysis/6xQp...,182053,4,00:03:02
6,1uddOsj7TyRA13hnS2yDyk,2bMqAaEuEUDHBkdTNAUl1f,21MZoYDTcbPsC5crOQVqho,Can't Take My Eyes Off You,Can't Take My Eyes Off You / 3BadSoSad - Edit,Private Island,47,2019-11-15,0.614,0.6090,...,0.1520,0.429,123.996,audio_features,spotify:track:1uddOsj7TyRA13hnS2yDyk,https://api.spotify.com/v1/tracks/1uddOsj7TyRA...,https://api.spotify.com/v1/audio-analysis/1udd...,209000,4,00:03:29
7,2nUV1fiD45RN6cQZ85GDc1,0lvgNBfe5wm9gKkecWN5Hq,18qigp8zUYz9THL4t4vZV4,Salt (Nlmg),The Light Left Over,Ben Hon,39,2019-04-05,0.570,0.5950,...,0.1420,0.336,136.588,audio_features,spotify:track:2nUV1fiD45RN6cQZ85GDc1,https://api.spotify.com/v1/tracks/2nUV1fiD45RN...,https://api.spotify.com/v1/audio-analysis/2nUV...,202840,4,00:03:22
8,3FRJFImdfX5NSY3QH3jI4u,4cZhiGqIqqqlHxWOC9r7Jp,7FKTg75ADVMZgY3P9ZMRtH,Glistening,Something You Needed,Flipturn,37,2020-02-07,0.668,0.5740,...,0.1150,0.248,127.011,audio_features,spotify:track:3FRJFImdfX5NSY3QH3jI4u,https://api.spotify.com/v1/tracks/3FRJFImdfX5N...,https://api.spotify.com/v1/audio-analysis/3FRJ...,247559,4,00:04:07
9,3flgdcFBWI84DPi4s1jhhd,4cZhiGqIqqqlHxWOC9r7Jp,7FKTg75ADVMZgY3P9ZMRtH,Savannah,Something You Needed,Flipturn,30,2020-02-07,0.535,0.0755,...,0.2380,0.409,104.443,audio_features,spotify:track:3flgdcFBWI84DPi4s1jhhd,https://api.spotify.com/v1/tracks/3flgdcFBWI84...,https://api.spotify.com/v1/audio-analysis/3flg...,69072,4,00:01:09


In [36]:
master_df.to_excel(writer,'top_tracks')

# Create Excel File

In [37]:
current_directory = os.path.abspath(os.getcwd())

In [38]:
export_file_path = os.path.join(current_directory, 'spotify_export_files', file_name)

In [39]:
# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter(export_file_path, engine='xlsxwriter')

# Convert the dataframe to an XlsxWriter Excel object.
df1.to_excel(writer,'recently_played', index=False)
master_df.to_excel(writer,'top_tracks',index=False)

# Get the xlsxwriter workbook and worksheet objects.
workbook  = writer.book
worksheet1 = writer.sheets['recently_played']
worksheet2 = writer.sheets['top_tracks']


for i, col in enumerate(master_df.columns):
    # find length of column i
    column_len = master_df[col].astype(str).str.len().max()
    # Setting the length if the column header is larger
    # than the max column value length
    column_len = max(column_len, len(col)) + 2
    
    # set the column length
    worksheet1.set_column(i, i, column_len)
    
for i, col in enumerate(df1.columns):
    # find length of column i
    column_len = df1[col].astype(str).str.len().max()
    # Setting the length if the column header is larger
    # than the max column value length
    column_len = max(column_len, len(col)) + 2
    
    # set the column length
    worksheet2.set_column(i, i, column_len)

writer.save()

# Create Playlist

In [ ]:
username = config.username
client_id = config.client_id
client_secret = config.client_secret
redirect_uri = 'http://localhost:7777/callback'
scope = 'playlist-modify-private'

auth_token = util.prompt_for_user_token(username=username, 
                                   scope=scope, 
                                   client_id=client_id,   
                                   client_secret=client_secret,     
                                   redirect_uri=redirect_uri)

In [ ]:
base_url = f'https://api.spotify.com/v1/users/{username}/playlists'

#2. Authentication
#3. Parameters -- would be stored with authentication
headers = {
    "Content-Type":"application/json",
    "Authorization": f"Bearer {auth_token}"
}

request_body = json.dumps({
          "name": "Test Python Playlist",
          "description": "This is a test playlist generated by Python",
          "public": False # private
        })
#4. Create an empty list
r = requests.post(url = base_url, data = request_body, headers=headers)
r.text

# Pull Playlist ID

In [40]:
username = config.username
client_id = config.client_id
client_secret = config.client_secret
redirect_uri = 'http://localhost:7777/callback'
scope = 'playlist-read-private'

auth_token = util.prompt_for_user_token(username=username, 
                                   scope=scope, 
                                   client_id=client_id,   
                                   client_secret=client_secret,     
                                   redirect_uri=redirect_uri)

In [41]:
base_url = f'https://api.spotify.com/v1/users/{username}/playlists?'

#2. Authentication
#3. Parameters -- would be stored with authentication
headers = {
    "Authorization": f"Bearer {auth_token}"
}

#4. Create an empty list
personal_playlist_data = [] #would be good explore how to capture data at different points in time
r = requests.get(base_url+"&limit=50", headers=headers)
personal_playlist_data.append(json.loads(r.text))

In [42]:
playlist_ids = []
playlist_names = []
playlist_descriptions = []
playlist_owners = []
playlist_publics = []

for i in range(len(personal_playlist_data[0]['items'])):
    playlist_ids.append(personal_playlist_data[0]['items'][i]['id'])
    playlist_names.append(personal_playlist_data[0]['items'][i]['name'])
    playlist_descriptions.append(personal_playlist_data[0]['items'][i]['description'])
    playlist_owners.append(personal_playlist_data[0]['items'][i]['owner']['display_name'])
    playlist_publics.append(personal_playlist_data[0]['items'][i]['public'])

In [43]:
list_dic4={'playlist_id':playlist_ids,
           'playlist_name':playlist_names,
           'playlist_description':playlist_descriptions,
           'playlist_owner':playlist_owners,
           'playlist_public':playlist_publics
    }

In [45]:
playlist_df = pd.DataFrame(list_dic4)
playlist_df.head()

,playlist_id,playlist_name,playlist_description,playlist_owner,playlist_public
0,79tXL4BD7MnqvpMZtKDho9,Test Python Playlist,This is a test playlist generated by Python,Nicholas Drake,False
1,5b4Ko7aAOpk8JT9qQnngOy,Pop Punk,,Amber Layn Miller,False
2,0RSKurPBF9Jd2W6zjAw6ZI,Chicago Unite At Night Complete Collection,Chicago nightly sing-along collection during C...,Chicago Unite At Night,False
3,7w3yeYpdXvclEmxFfsgVNr,Holiday-ish (feat. Dylan Minnette),,Nicholas Drake,True
4,7fdwWjSOaHB3UBBKAyXZLJ,Bill burr podcasts,,cikik_55,False


# Post song to playlist

In [46]:
username = config.username
client_id = config.client_id
client_secret = config.client_secret
redirect_uri = 'http://localhost:7777/callback'
scope = 'playlist-modify-public, playlist-modify-private'

auth_token = util.prompt_for_user_token(username=username, 
                                   scope=scope, 
                                   client_id=client_id,   
                                   client_secret=client_secret,     
                                   redirect_uri=redirect_uri)

In [47]:
playlist_id = '79tXL4BD7MnqvpMZtKDho9'
base_url = f"https://api.spotify.com/v1/playlists/{playlist_id}/tracks"

# Must be in list to load into playlist
uris = ['spotify:track:4pvb0WLRcMtbPGmtejJJ6y']

# Authentication
# Parameters -- would be stored with authentication
headers = {
    "Content-Type":"application/json",
    "Authorization": f"Bearer {auth_token}"
}

request_body = json.dumps({
          "uris":uris
        })
# Make request
r = requests.post(url = base_url, data = request_body, headers=headers)
r.text

'{\n  "snapshot_id" : "NCw4YmM5ZjUxNzRjZmI4NGEzMWM1Yzk0NGFmODAyNjAyZDBkOGI4ZWFm"\n}'